In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_ikan = pd.read_csv('/content/drive/MyDrive/csv/dataset_teman_ikan_final.csv')

In [6]:
df_ikan.head(5)

,id_ikan,nama_ikan,ukuran_maksimum_cm,min_suhu_optimal_celcius,max_suhu_optimal_celcius,min_suhu_pH_air,max_suhu_pH_air,sifat_ikan
0,0,Betta Splendens (Cupang),7,24,28,6.0,7.0,1
1,1,Red Devil Cichlid (Amphilophus labiatus),30,25,28,6.5,7.5,2
2,2,Guppy (Poecilia reticulata),4,22,28,6.5,8.0,0
3,3,Komet (Carrasius),20,27,30,6.0,8.3,0
4,4,Manfish (Pterophyllum scalare),50,25,28,6.0,8.0,1


In [7]:
label_encoder = LabelEncoder()
df_ikan['sifat_ikan'] = label_encoder.fit_transform(df_ikan['sifat_ikan'])

In [8]:
scaler = MinMaxScaler()
numeric_features = ['ukuran_maksimum_cm', 'min_suhu_optimal_celcius', 'max_suhu_optimal_celcius', 'min_suhu_pH_air', 'max_suhu_pH_air']
df_ikan[numeric_features] = scaler.fit_transform(df_ikan[numeric_features])

In [9]:
def build_model():
    input_numeric = Input(shape=(len(numeric_features),))
    input_sifat = Input(shape=(1,))

    # Add a dense layer for numeric features
    dense_numeric = Dense(64, activation='relu')(input_numeric)

    # Embed 'sifat_ikan' using an embedding layer
    embed_sifat = Embedding(input_dim=len(label_encoder.classes_), output_dim=10)(input_sifat)
    flatten_sifat = Flatten()(embed_sifat)

    # Concatenate the outputs of numeric and embedded layers
    merged = Concatenate()([dense_numeric, flatten_sifat])

    # Output for 'sifat_ikan'
    output_sifat = Dense(1, activation='sigmoid', name='output_sifat')(merged)

    # Output for 'nama_ikan'
    output_nama_ikan = Dense(len(df_ikan), activation='softmax', name='output_nama_ikan')(flatten_sifat)

    model = Model(inputs=[input_numeric, input_sifat], outputs=[output_sifat, output_nama_ikan])
    model.compile(optimizer='adam', loss={'output_sifat': 'binary_crossentropy', 'output_nama_ikan': 'categorical_crossentropy'}, metrics=['accuracy'])
    return model

In [10]:
model = build_model()
model.fit(x=[df_ikan[numeric_features],
             df_ikan['sifat_ikan']],
          y={'output_sifat': df_ikan['sifat_ikan'],
             'output_nama_ikan': pd.get_dummies(df_ikan['nama_ikan'])},
          epochs=10, batch_size=32)

Epoch 1/10
1/1 [==============================] - 1s 969ms/step - loss: 3.8361 - output_sifat_loss: 0.7455 - output_nama_ikan_loss: 3.0906 - output_sifat_accuracy: 0.4091 - output_nama_ikan_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 3.8249 - output_sifat_loss: 0.7347 - output_nama_ikan_loss: 3.0902 - output_sifat_accuracy: 0.4091 - output_nama_ikan_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 3.8138 - output_sifat_loss: 0.7240 - output_nama_ikan_loss: 3.0898 - output_sifat_accuracy: 0.4091 - output_nama_ikan_accuracy: 0.0455
Epoch 4/10
1/1 [==============================] - 0s 10ms/step - loss: 3.8030 - output_sifat_loss: 0.7136 - output_nama_ikan_loss: 3.0894 - output_sifat_accuracy: 0.4091 - output_nama_ikan_accuracy: 0.0455
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 3.7922 - output_sifat_loss: 0.7032 - output_nama_ikan_loss: 3.0890 - output_sifat_accuracy: 0.409

In [11]:
def get_recommendations(id_ikan):
    input_numeric = df_ikan.loc[id_ikan, numeric_features].values.reshape(1, -1)
    input_sifat = df_ikan.loc[id_ikan, 'sifat_ikan']

    # Ensure that input data types are correct
    input_numeric = input_numeric.astype(np.float32)
    input_sifat = np.array([input_sifat]).astype(np.int32)

    # Predict using the trained model
    predictions = model.predict([input_numeric, input_sifat])

    # Extract indices of top 3 predictions (excluding the input itself)
    output_sifat_prob = predictions[0][0]
    output_nama_ikan_prob = predictions[1][0]

    # Get the top 3 predicted classes for 'nama_ikan'
    top_classes = np.argsort(output_nama_ikan_prob)[::-1][:3]

    # Filter recommendations based on 'sifat_ikan'
    if input_sifat == 0:
        filtered_indices = np.intersect1d(np.where(df_ikan['sifat_ikan'].isin([0, 1])), top_classes)
    elif input_sifat == 1:
        filtered_indices = np.intersect1d(np.where(df_ikan['sifat_ikan'].isin([0, 1])), top_classes)
    elif input_sifat == 2:
        filtered_indices = np.intersect1d(np.where(df_ikan['sifat_ikan'].isin([1, 2])), top_classes)

    # Get the top 3 similar fish
    top_similar_ikan = df_ikan.iloc[filtered_indices][['nama_ikan']]

    return top_similar_ikan

In [12]:
id_ikan_input = 3  # Replace with the desired id_ikan
recommendations = get_recommendations(id_ikan_input)
print(recommendations)

1/1 [==============================] - 0s 70ms/step
                        nama_ikan
6   glofish (Poecilia reticulata)
13  Arwana (Scleropages formosus)


In [ ]:
model.save('/content/drive/MyDrive/csv/model_rekomendasi/model_rekomendasi_v3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
